In [1]:
import numpy as np 
import pandas as pd 
import os 
import pickle 

## Remove the empty files from the KSC MERLIN Cloud to Cloud Dataset

In [2]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
for yr in yrs:
    for mo in mos:
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            try:
                    temp = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            except:
                print('no data: '+file+' removing')
                os.remove(dir+file)


no data: KSCCC20170127.dat removing
no data: KSCCC20170126.dat removing
no data: KSCCC20170130.dat removing
no data: KSCCC20170124.dat removing
no data: KSCCC20170118.dat removing
no data: KSCCC20170119.dat removing
no data: KSCCC20170125.dat removing
no data: KSCCC20170131.dat removing
no data: KSCCC20170109.dat removing
no data: KSCCC20170121.dat removing
no data: KSCCC20170120.dat removing
no data: KSCCC20170112.dat removing
no data: KSCCC20170113.dat removing
no data: KSCCC20170111.dat removing
no data: KSCCC20170110.dat removing
no data: KSCCC20170128.dat removing
no data: KSCCC20170114.dat removing
no data: KSCCC20170101.dat removing
no data: KSCCC20170115.dat removing
no data: KSCCC20170129.dat removing
no data: KSCCC20170103.dat removing
no data: KSCCC20170117.dat removing
no data: KSCCC20170116.dat removing
no data: KSCCC20170102.dat removing
no data: KSCCC20170226.dat removing
no data: KSCCC20170225.dat removing
no data: KSCCC20170224.dat removing
no data: KSCCC20170220.dat r

In [5]:
first_file = True
for yr in yrs:
    for mo in mos:
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            if first_file==True:
                first_file=False
                merlin_cc = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            else:
                merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False))

/var/folders/nk/vl3n0lp13k7b3tt0mx4qkydh0000gn/T/ipykernel_20617/351914617.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False))
/var/folders/nk/vl3n0lp13k7b3tt0mx4qkydh0000gn/T/ipykernel_20617/351914617.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False))
/var/folders/nk/vl3n0lp13k7b3tt0mx4qkydh0000gn/T/ipykernel_20617/351914617.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,